![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate%2FConcepts&file=Targeted+Document+Extraction.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Concepts/Targeted%20Document%20Extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FConcepts%2FTargeted%2520Document%2520Extraction.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Concepts/Targeted%20Document%20Extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Concepts/Targeted%20Document%20Extraction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Targeted Document Extraction

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.genai', 'google-genai'),
    ('pydantic', 'pydantic'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [6]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [7]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [8]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [9]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'concept-doc-extract'

packages:

In [31]:
# standard packages
import os, shutil, enum, json, base64, typing

import IPython
import pydantic

# google gen ai sdk
from google import genai

In [11]:
genai.__version__

'1.2.0'

clients:

In [12]:
genai_client = genai.Client(vertexai = True, project = PROJECT_ID, location = REGION)

---
## Multimodal Inputs

Prompts can extend beyond text to include audio, video, images, and documents as context.  This means you can [design multimodal prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/design-multimodal-prompts) that include combinations of these file types as grounding for Gemini.

### Get The Documents

A set of example file inputs for multimodal prompts were created in this companion workflow: [Create Files For Multimodal Prompt Examples](../Generate/Create%20Files%20For%20Multimodal%20Prompt%20Examples.ipynb) and the outputs are already included in the repository.

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [13]:
local_dir = '../files/multimodal-inputs'

In [14]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Generate/files/multimodal-inputs', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../files/multimodal-inputs`


---
### **Prompt With Documents**

[Document Understanding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/document-understanding)

#### Review The Document

In [21]:
IPython.display.IFrame(f"{local_dir}/invoice.pdf", width=800, height= 800)

#### Read The Document Data

In [22]:
with open(f"{local_dir}/invoice.pdf", 'rb') as f:
    document_read = f.read()

#### Inline Loading

In [23]:
document_inline = genai.types.Part.from_bytes(
    data = base64.b64encode(document_read).decode('utf_8'),
    mime_type = 'application/pdf'
)

#### Generation Example

In [25]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [document_inline, 'Describe the content of this invoice.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0.3
    )
)

IPython.display.Markdown(response.text)

Here's a description of the invoice content:

**General Information:**

*   **Invoice Number:** 20231027-001
*   **Company:** Acme Data Visualization
*   **Company Address:** 123 Main Street, Anytown, CA 91234
*   **Company Phone:** (555) 123-4567
*   **Company Email:** info@acmedatavis.com
*   **Invoice Date:** October 27, 2023
*   **Due Date:** November 26, 2023

**Items/Services:**

*   Plot Visualization (.png, .jpeg, .webp): Quantity 1, Rate $150.00, Amount $150.00
*   Animated Plot Visualization (.mp4): Quantity 1, Rate $200.00, Amount $200.00
*   Process Script & Audio (.mp3): Quantity 1, Rate $100.00, Amount $100.00

**Financial Summary:**

*   **Subtotal:** $450.00
*   **Sales Tax (6%):** $27.00
*   **Total:** $477.00

**Additional Notes:**

*   A note at the bottom requests payment to the address above and expresses gratitude for the business.

---
## Controlled Responses With `response_schema`

### Classify Document

In [35]:
class Invoices(str, enum.Enum):
    INVOICE = 'Invoice'
    OTHER = 'Not an invoice!'

In [29]:
genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [document_inline, 'Is this an invoice?'],
    config = genai.types.GenerateContentConfig(
        temperature = 0,
        system_instruction = 'Classify documents into categories.',
        response_mime_type = 'text/x.enum',
        response_schema = Invoices
    )
).text

'Invoice'

### Targeted Extraction With Schema

In [32]:
class InvoiceData(pydantic.BaseModel):
    id: str = pydantic.Field(description = 'The unique invoice number or identifier.')
    company_name: typing.Optional[str] = pydantic.Field(description = 'The name of the company issuing the invoice.')
    line_item_total: typing.List[float] = pydantic.Field(description = 'A list of the total amount for each line item on the invoice.')
    total_amount: float = pydantic.Field(description = 'The overall total amount due on the invoice.')

In [34]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [document_inline, 'Extract the requested information from this invoice.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0,
        system_instruction = 'Extract targeted information from invoices with high accuracy.',
        response_mime_type = 'application/json',
        response_schema = InvoiceData
    )
)

json.loads(response.text)

{'id': '20231027-001',
 'company_name': 'Acme Data Visualization',
 'line_item_total': [150.0, 200.0, 100.0],
 'total_amount': 477.0}